# Advent of code 2020 - day 11 - part 1

--- Day 11: Seating System ---
Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
Otherwise, the seat's state does not change.
Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
After a second round, the seats with four or more occupied adjacent seats become empty again:

#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
This process continues for three more rounds:

#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [1]:
def get_occupied_count(cur_pos, prev_line, cur_line, next_line, verbose = False):    
    
    count = 0
    positions = []
    max_pos = len(cur_line) - 1
    
    if (verbose):
        print(f"\ncur_pos: {cur_pos}")
    
    if cur_pos == 0:
        positions = [cur_pos, cur_pos + 1]
    elif cur_pos == max_pos:
        positions = [max_pos - 1, max_pos]
    else:
        positions = [cur_pos - 1, cur_pos, cur_pos + 1]
    
    if (verbose):
        print(f"\nmax_pos cur_line: {max_pos}, positions: {positions}")
    
    # previous line
    if prev_line is not None:
        for p in positions:
            if prev_line[p] == '#':
                count += 1
    
    # current line
    if cur_pos == 0:
        if cur_line[positions[-1]] == '#':
            count += 1  
        
    elif cur_pos == max_pos:
        if cur_line[positions[0]] == '#':
            count += 1 
        
    else:
        
        for p in [positions[0], positions[-1]]:  
            if cur_line[p] == '#':
                count += 1      
    
    # next line            
    if next_line is not None:
        for p in positions:
            if next_line[p] == '#':
                count += 1
    if (verbose):
        print(f"count: {count}")    
    return count

In [2]:
def apply_rules(lines, verbose = False):

    prev_line = None
    cur_line = None
    next_line = None
    line_count = len(lines)
    
    new_lines = []
    cur_count = 0 # number of occupied seats after this round

    for i in range(line_count):
        cur_line = lines[i]
        if (i == 0): # first line
            prev_line = None            
            next_line = lines[i+1]
        elif (i == line_count - 1): # last line
            prev_line = lines[i-1]
            next_line = None
        else:
            prev_line = lines[i-1]
            next_line = lines[i+1]
            
        if (verbose):
            print(f"\ni: {i}, \nprev_line: {prev_line}, \ncur_line: {cur_line}, \nnext_line: {next_line}")
            print(f"len(cur_line): {len(cur_line)}")
        
        new_line = cur_line.copy()
        for j, state in enumerate(cur_line):
            occ_count = get_occupied_count(j, prev_line, cur_line, next_line, verbose)
            if state == 'L' and occ_count == 0:
                new_line[j] = '#'
                cur_count += 1
            elif state == '#' and occ_count >= 4:
                new_line[j] = 'L'
            elif state == '#':
                cur_count += 1
        
        new_lines.append(new_line)
                
                
    assert len(lines) == len(new_lines)
    if (verbose):
        print(f"lines[0]: {lines[0]}, new_lines[0]: {new_lines[0]}")
    assert len(lines[0]) == len(new_lines[0])
    
    return cur_count, new_lines

In [3]:
def get_lines(filename):
    with open(filename, 'r', encoding = 'UTF-8') as file:
        lines = file.readlines()
    vals = []
    for i, x in enumerate(lines):        
        y = [val for val in x[:-1]] # strip out last newline char
        vals.append(y)
    return vals

In [4]:
lines = get_lines('input.txt')

In [5]:
prev_count = -1
it_counter = 1
cur_count, new_state = apply_rules(lines, verbose = False)

print(f"\nit_counter: {it_counter}, cur_count: {cur_count}, prev_count: {prev_count}")
    
while prev_count != cur_count:
    prev_count = cur_count
    cur_count, new_state = apply_rules(new_state)
    it_counter += 1
    
    print(f"\nit_counter: {it_counter}, cur_count: {cur_count}, prev_count: {prev_count}")


it_counter: 1, cur_count: 7431, prev_count: -1

it_counter: 2, cur_count: 122, prev_count: 7431

it_counter: 3, cur_count: 7072, prev_count: 122

it_counter: 4, cur_count: 245, prev_count: 7072

it_counter: 5, cur_count: 6727, prev_count: 245

it_counter: 6, cur_count: 354, prev_count: 6727

it_counter: 7, cur_count: 6452, prev_count: 354

it_counter: 8, cur_count: 455, prev_count: 6452

it_counter: 9, cur_count: 6201, prev_count: 455

it_counter: 10, cur_count: 570, prev_count: 6201

it_counter: 11, cur_count: 5931, prev_count: 570

it_counter: 12, cur_count: 696, prev_count: 5931

it_counter: 13, cur_count: 5668, prev_count: 696

it_counter: 14, cur_count: 804, prev_count: 5668

it_counter: 15, cur_count: 5421, prev_count: 804

it_counter: 16, cur_count: 897, prev_count: 5421

it_counter: 17, cur_count: 5216, prev_count: 897

it_counter: 18, cur_count: 989, prev_count: 5216

it_counter: 19, cur_count: 5005, prev_count: 989

it_counter: 20, cur_count: 1068, prev_count: 5005

it_count

In [6]:
print(f"After {it_counter} iterations, the number of occupied seats stabilised at {cur_count}.")

After 94 iterations, the number of occupied seats stabilised at 2275.


# Part 2

--- Part Two ---
As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

.............
.L.L.#.#.#.#.
.............
The empty seat below would see no occupied seats:

.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?